## Part 4: Fraud

In [4]:
%run "../scripts/ETL.py"

22/09/21 10:27:00 WARN Utils: Your hostname, Aishwaryas-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.12.134.13 instead (on interface en0)
22/09/21 10:27:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/21 10:27:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/21 10:27:00 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/21 10:27:00 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/09/21 10:27:00 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/09/21 10:27:00 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/09/21 10:27:00 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.


AnalysisException: Cannot resolve column name "postcodes" among (id, postcode, locality, state, long, lat, dc, type, status, sa3, sa3name, sa4, sa4name, region, Lat_precise, Long_precise, SA1_MAINCODE_2011, SA1_MAINCODE_2016, SA2_MAINCODE_2016, SA2_NAME_2016, SA3_CODE_2016, SA3_NAME_2016, SA4_CODE_2016, SA4_NAME_2016, RA_2011, RA_2016, MMM_2015, MMM_2019, ced, altitude, chargezone, phn_code, phn_name, lgaregion, electorate, electoraterating)

The consumer fraud data shows the list of transactions with a high probability of being fraudulent, and therefore should be considered invalid.

In [5]:
print(fraud_consumer.count()) 
fraud_consumer.limit(5)

34864


user_id,order_datetime,fraud_probability
6228,2021-12-19,97.6298077657765
21419,2021-12-10,99.24738020302328
5606,2021-10-17,84.05825045251777
3101,2021-04-17,91.42192091901347
22239,2021-10-19,94.70342477508035


Probability of fraud for high revenue merchants...?

In [6]:
print(fraud_merchants.count())
fraud_merchants.limit(5)

114


merchant_abn,order_datetime,fraud_probability
19492220327,2021-11-28,44.403658647495355
31334588839,2021-10-02,42.75530083865367
19492220327,2021-12-22,38.867790051131095
82999039227,2021-12-19,94.1347004808891
90918180829,2021-09-02,43.32551731714902


In [7]:
final_join.limit(5)

merchant_name,merchant_abn,categories,take_rate,revenue_levels,name,address,state,postcode,gender,trans_merchant_abn,dollar_value,order_id,order_datetime,user_id,consumer_id
Felis Limited,10023283211,"furniture, home f...",0.18,e,Roberto Robbins,3146 Eric Turnpik...,NSW,2396,Male,10023283211,74.68707218671928,b36e56d0-dbb2-479...,2021-08-11,23679,6664
Felis Limited,10023283211,"furniture, home f...",0.18,e,Kelly Krueger,039 Green Route,WA,6935,Undisclosed,10023283211,319.4986425058244,51798dc0-8a7b-4cd...,2021-12-28,2199,8509
Felis Limited,10023283211,"furniture, home f...",0.18,e,Brianna Hart,850 Joshua Viaduc...,WA,6415,Female,10023283211,123.61804677878001,edc9c5b1-82a0-4ee...,2021-10-13,19278,15265
Felis Limited,10023283211,"furniture, home f...",0.18,e,Jose Frye,394 Matthews View...,NSW,2021,Male,10023283211,191.63853773963905,376cc026-5d91-498...,2021-07-04,5156,19115
Felis Limited,10023283211,"furniture, home f...",0.18,e,Matthew Waters,729 Harper Spring...,WA,6253,Male,10023283211,250.13111867397248,0220b2de-768c-49e...,2022-07-09,23774,20250


- join consumer fraud to final join
- create a version of final join with removed transactions from fraud
- aggregate final join from transaction to merchant level
- add fraud merchant as a feature to aggregated final join
- treat fraud as response and try to create model to give other merchants a fraud score

In [8]:
fj_with_fraud = final_join.join(fraud_consumer, (final_join.user_id == fraud_consumer.user_id) \
                               & (final_join.order_datetime == fraud_consumer.order_datetime), "left_outer")
fj_with_fraud.limit(5)

merchant_name,merchant_abn,categories,take_rate,revenue_levels,name,address,state,postcode,gender,trans_merchant_abn,dollar_value,order_id,order_datetime,user_id,consumer_id,user_id,order_datetime,fraud_probability
Felis Limited,10023283211,"furniture, home f...",0.18,e,Roberto Robbins,3146 Eric Turnpik...,NSW,2396,Male,10023283211,74.68707218671928,b36e56d0-dbb2-479...,2021-08-11,23679,6664,null,null,null
Felis Limited,10023283211,"furniture, home f...",0.18,e,Kelly Krueger,039 Green Route,WA,6935,Undisclosed,10023283211,319.4986425058244,51798dc0-8a7b-4cd...,2021-12-28,2199,8509,null,null,null
Felis Limited,10023283211,"furniture, home f...",0.18,e,Brianna Hart,850 Joshua Viaduc...,WA,6415,Female,10023283211,123.61804677878001,edc9c5b1-82a0-4ee...,2021-10-13,19278,15265,null,null,null
Felis Limited,10023283211,"furniture, home f...",0.18,e,Jose Frye,394 Matthews View...,NSW,2021,Male,10023283211,191.63853773963905,376cc026-5d91-498...,2021-07-04,5156,19115,null,null,null
Felis Limited,10023283211,"furniture, home f...",0.18,e,Matthew Waters,729 Harper Spring...,WA,6253,Male,10023283211,250.13111867397248,0220b2de-768c-49e...,2022-07-09,23774,20250,null,null,null


In [9]:
fj_rem_fraud = fj_with_fraud.filter("fraud_probability is NULL")
fj_rem_fraud.count()

14115157

In [10]:
user_details.count()

499999

In [11]:
final_join.count()

14195505

In [12]:
tbl_merchants.limit(5)

merchant_name,merchant_abn,categories,take_rate,revenue_levels
Felis Limited,10023283211,"furniture, home f...",0.18,e
Arcu Ac Orci Corp...,10142254217,"cable, satellite,...",4.22,b
Nunc Sed Company,10165489824,"jewelry, watch, c...",4.4,b
Ultricies Digniss...,10187291046,"wAtch, clock, and...",3.29,b
Enim Condimentum PC,10192359162,music shops - mus...,6.33,a


In [16]:
fj_rem_fraud = fj_rem_fraud.withColumnRenamed("fraud_probability","cons_fraud_prob")
fj_rem_fraud = fj_rem_fraud.withColumnRenamed("merchant_name","merchant_name_1")
fj_rem_fraud = fj_rem_fraud.withColumnRenamed("merchant_abn","merchant_abn_1")
fj_rem_fraud = fj_rem_fraud.withColumnRenamed("take_rate","take_rate_1")
fj_rem_fraud = fj_rem_fraud.withColumnRenamed("categories","categories_1")
fj_rem_fraud = fj_rem_fraud.withColumnRenamed("revenue_levels","revenue_levels_1")

# fj_rem_fraud.limit(5)

In [17]:
#Joining merchants fraud table to large table 

from pyspark.sql import functions as F


fj_rem_fraud.createOrReplaceTempView("temp1")

tbl_merchants.createOrReplaceTempView("temp2")

joined_data = spark.sql(""" 

SELECT *
FROM temp1

INNER JOIN temp2

ON temp1.merchant_abn_1 = temp2.merchant_abn 
AND temp1.merchant_name_1 = temp2.merchant_name 
AND temp1.take_rate_1 = temp2.take_rate 
AND temp1.revenue_levels_1 = temp2.revenue_levels 
""")
print("join done")

join done


In [18]:
joined_data.withColumnRenamed("fraud_probability","merchant_fraud_prob")
joined_data.limit(5) 



merchant_name_1,merchant_abn_1,categories_1,take_rate_1,revenue_levels_1,name,address,state,postcode,gender,trans_merchant_abn,dollar_value,order_id,order_datetime,user_id,consumer_id,user_id,order_datetime,cons_fraud_prob,merchant_name,merchant_abn,categories,take_rate,revenue_levels
Felis Limited,10023283211,"furniture, home f...",0.18,e,Roberto Robbins,3146 Eric Turnpik...,NSW,2396,Male,10023283211,74.68707218671928,b36e56d0-dbb2-479...,2021-08-11,23679,6664,null,null,null,Felis Limited,10023283211,"furniture, home f...",0.18,e
Felis Limited,10023283211,"furniture, home f...",0.18,e,Kelly Krueger,039 Green Route,WA,6935,Undisclosed,10023283211,319.4986425058244,51798dc0-8a7b-4cd...,2021-12-28,2199,8509,null,null,null,Felis Limited,10023283211,"furniture, home f...",0.18,e
Felis Limited,10023283211,"furniture, home f...",0.18,e,Brianna Hart,850 Joshua Viaduc...,WA,6415,Female,10023283211,123.61804677878001,edc9c5b1-82a0-4ee...,2021-10-13,19278,15265,null,null,null,Felis Limited,10023283211,"furniture, home f...",0.18,e
Felis Limited,10023283211,"furniture, home f...",0.18,e,Jose Frye,394 Matthews View...,NSW,2021,Male,10023283211,191.63853773963905,376cc026-5d91-498...,2021-07-04,5156,19115,null,null,null,Felis Limited,10023283211,"furniture, home f...",0.18,e
Felis Limited,10023283211,"furniture, home f...",0.18,e,Matthew Waters,729 Harper Spring...,WA,6253,Male,10023283211,250.13111867397248,0220b2de-768c-49e...,2022-07-09,23774,20250,null,null,null,Felis Limited,10023283211,"furniture, home f...",0.18,e


In [19]:
model_data = joined_data.drop("merchant_name_1").drop("merchant_abn_1").drop("categories_1").drop("take_rate_1").drop("revenue_levels_1").drop("name").drop("address").drop("state").drop("gender")

In [20]:
model_data.limit(5)

postcode,trans_merchant_abn,dollar_value,order_id,order_datetime,user_id,consumer_id,user_id,order_datetime,cons_fraud_prob,merchant_name,merchant_abn,categories,take_rate,revenue_levels
3862,10023283211,69.90225010580168,d6f41efc-4c08-455...,2021-05-21,1859,797884,null,null,null,Felis Limited,10023283211,"furniture, home f...",0.18,e
1157,10023283211,277.5959492940812,6884dcd0-e2c0-4f3...,2021-08-22,19083,1250662,null,null,null,Felis Limited,10023283211,"furniture, home f...",0.18,e
2619,10023283211,218.0471981318539,1da32311-ab3f-4ef...,2021-08-27,14673,349420,null,null,null,Felis Limited,10023283211,"furniture, home f...",0.18,e
4497,10023283211,429.13905563051316,1e704bea-f123-4fa...,2021-05-21,10932,1412788,null,null,null,Felis Limited,10023283211,"furniture, home f...",0.18,e
2087,10023283211,105.48434125775125,fc4f153e-9667-44e...,2021-08-21,18249,1314551,null,null,null,Felis Limited,10023283211,"furniture, home f...",0.18,e


In [ ]:
model_data = model_data.select("consumer_id", "cons_fraud_prob")

# model_data_pdf = model_data.toPandas()

22/09/21 11:23:35 WARN TaskMemoryManager: Failed to allocate a page (33554432 bytes), try again.


22/09/21 11:24:26 WARN TaskMemoryManager: Failed to allocate a page (33554432 bytes), try again.
22/09/21 11:24:26 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 230219 ms exceeds timeout 120000 ms


22/09/21 11:25:30 WARN TaskMemoryManager: Failed to allocate a page (33554432 bytes), try again.


In [ ]:
model_data.dropna()

In [ ]:
# cons_clusters = model_data_pdf["cons_fraud_prob", "dollar_value"]

In [24]:
model_data_pdf = model_data.toPandas()


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/ahirve/opt/anaconda3/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/ahirve/opt/anaconda3/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/ahirve/opt/anaconda3/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
import sklearn 
from sklearn.mixture import GMM
gmm = GMM(n_components=4).fit(model_data_pdf["cons_fraud_prob"])
labels = gmm.predict(model_data_pdf["cons_fraud_prob"])
plt.scatter(model_data_pdf["cons_fraud_prob"][:, 0], model_data_pdf["cons_fraud_prob"][:, 1], c=labels, s=40, cmap='viridis');